# Simulation #2
Having written the functions required to implement estimation of $T$, we now experiment with a few different model parameters.

In [1]:
using JLD

In [53]:
include("functions.jl");

## The setup
As before, we set up the state space as the flat torus $\mathbb{T}^2$, represented by the square $[0, 2\pi] \times [0, 2\pi]$.

In [ ]:
range = 2π;

In [48]:
grid_size = 100;
n_gridpoints = grid_size ^ 2;
grid = creategrid(0, range, grid_size);

Throughout all of our experiments, we will use the same sample of $N=10000$ points drawn from the uniform distribution.

In [49]:
sample_size = 10000;
s = sampledist(sample_size, range);

## Experimentation
We can now run various experiments using different model parameters.

In [52]:
maps = ["standard", "cat", "cat-perturbed"];
epsilons = [0.01, 0.1, 1, 10];
n_bases = [5, 10, 25, 33];

In [ ]:
estimate_T(grid, s, basis_grid_size, ϵ; map_type)

## Varying the integral resolution
As an initial test, I don't expect that varying the resolution of the integral calculations should have a significant effect on the calculation of $T$.

In [8]:
integral_resolutions = [100, 200, 500, 1000];
results = [];
for res in integral_resolutions
    T = estimate_T(state_space, 10000, S, φ, 0.9, 5, res);
    push!(results, T);
    println("Estimation complete for: $res")
end

Academic license - for non-commercial use only - expires 2021-08-05
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 25 rows, 10025 columns and 250025 nonzeros
Model fingerprint: 0xeb1c6616
Model has 10025 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-11, 1e+00]
  Objective range  [3e-02, 3e-02]
  QObjective range [2e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve time: 0.05s
Presolved: 25 rows, 10025 columns, 250025 nonzeros
Presolved model has 10025 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+02
 Factor NZ  : 3.250e+02 (roughly 4 MBytes of memory)
 Factor Ops : 5.525e+03 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.60492339e+09 -1.

Objective value is: 248.10203063778903
Estimation complete for: 1000


In [11]:
d = Dict(
    "results"=>results, 
    "integral_resolutions"=>integral_resolutions,
    "n_bases"=>5^2,
    "sample_size"=>10000,
    "epsilon"=>0.9,
    "S"=>"standard"
);

In [12]:
save("estimations/experiment_integral_resolutions.jld", "d", d)

In [15]:
for i in 1:4
    resolution = integral_resolutions[i];
    λ, Λ = ordered_eigendecomp(results[i]);
    println("With integral resolution $resolution, leading eigenvalue is: $(λ[1])")
end

With integral resolution 100, leading eigenvalue is 1.0038743696260968 + 0.0im
With integral resolution 200, leading eigenvalue is 1.0038752431912221 + 0.0im
With integral resolution 500, leading eigenvalue is 1.0038755363842837 + 0.0im
With integral resolution 1000, leading eigenvalue is 1.0038754057411512 + 0.0im


Clearly, varying the integral resolution has not affected the integrity of the estimation procedure. What about the estimate of the invariant density?

In [31]:
for i in 1:4
    resolution = integral_resolutions[i];
    λ, Λ = ordered_eigendecomp(results[i]);
    println("With integral resolution $resolution, leading eigenvector is:")
    println(Λ[:,1])
end

With integral resolution 100, leading eigenvector is:
ComplexF64[-0.19999896176010518 + 0.0im, -0.20000197086113408 + 0.0im, -0.19999569775471018 + 0.0im, -0.20000110003804084 + 0.0im, -0.19999932879284368 + 0.0im, -0.20000102906570572 + 0.0im, -0.20000028013626053 + 0.0im, -0.20000157240545194 + 0.0im, -0.20000231733741577 + 0.0im, -0.19999716542444523 + 0.0im, -0.19999723194306132 + 0.0im, -0.1999988250519022 + 0.0im, -0.200003244530273 + 0.0im, -0.19999594969310305 + 0.0im, -0.20000537917865047 + 0.0im, -0.20000136208533922 + 0.0im, -0.20000073924641526 + 0.0im, -0.19999712949005974 + 0.0im, -0.20000273932369675 + 0.0im, -0.19999854482808124 + 0.0im, -0.19999880468987752 + 0.0im, -0.19999991112462923 + 0.0im, -0.20000337598341675 + 0.0im, -0.19999841193429813 + 0.0im, -0.19999892696151506 + 0.0im]
With integral resolution 200, leading eigenvector is:
ComplexF64[0.19999850038855338 + 0.0im, 0.20000015568423318 + 0.0im, 0.20000322767852946 + 0.0im, 0.1999976905395623 + 0.0im, 0.200000

They are all almost the same. This is an encouraging validition of the integrity of our estimation procedure.

## Varying $\epsilon$
Varying the bandwidth of the basis functions however should have an effect, since this will by definition alter the structure of the subspace in which we are estimating $\mathcal{L}$.

In [37]:
epsilons = [0.01, 0.1, 1, 10];
results = [];
for epsilon in epsilons
    T = estimate_T(state_space, 10000, S, φ, epsilon, 33, 100);
    push!(results, T);
    println("Estimation complete for ϵ=$epsilon")
end

Academic license - for non-commercial use only - expires 2021-08-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1089 rows, 11089 columns and 3629 nonzeros
Model fingerprint: 0x8cd1e90b
Model has 11089 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [7e-04, 7e-04]
  QObjective range [2e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-04, 3e-04]
Presolve removed 118 rows and 7578 columns
Presolve time: 0.01s
Presolved: 971 rows, 3511 columns, 3511 nonzeros
Presolved model has 3511 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 9.710e+02 (roughly 2 MBytes of memory)
 Factor Ops : 9.710e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective      

Objective value is: 0.0016927364556502122
Estimation complete for ϵ=10.0


In [38]:
d = Dict(
    "results"=>results, 
    "epsilons"=>epsilons,
    "sample_size"=>10000,
    "S"=>"standard",
    "n_bases"=>33^2
);

In [39]:
save("estimations/experiment_epsilons.jld", "d", d)

In [41]:
for i in 1:4
    epsilon = epsilons[i]
    λ, Λ = ordered_eigendecomp(results[i]);
    println("With ϵ=$epsilon, leading eigenvalue is $(λ[1])")
end

With ϵ=0.01, leading eigenvalue is 0.014644405474734752 + 0.0im
With ϵ=0.1, leading eigenvalue is 1.011694057314656 + 0.0im
With ϵ=1.0, leading eigenvalue is 0.9999834162656183 + 0.0im
With ϵ=10.0, leading eigenvalue is 0.1177627848554035 + 0.0im


For at least a range of one order of magnitude, we get a reliable estimation procedure.

## Varying the dynamical map
I will try three different maps:
1. The standard map. This is known to have Lebesgue measure as the invariant density.
2. The cat map. This too should have a constant function as its invariant density.
3. The perturbed cat map. Even by perturbing the cat map only slightly, we should here obtain a non-constant function as the invariant density.

In [46]:
T_standard = estimate_T(state_space, 10000, S, φ, 1.0, 20, 100; map_type="standard");

Academic license - for non-commercial use only - expires 2021-08-05
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 400 rows, 10400 columns and 4000400 nonzeros
Model fingerprint: 0x09243d22
Model has 10400 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-09, 1e+00]
  Objective range  [2e-03, 2e-03]
  QObjective range [2e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve time: 0.89s
Presolved: 400 rows, 10400 columns, 4000400 nonzeros
Presolved model has 10400 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.980e+04
 Factor NZ  : 8.020e+04 (roughly 5 MBytes of memory)
 Factor Ops : 2.141e+07 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.69866670e+09

In [47]:
T_cat = estimate_T(state_space, 10000, S, φ, 1.0, 20, 100; map_type="cat");

Academic license - for non-commercial use only - expires 2021-08-05
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 400 rows, 10400 columns and 4000400 nonzeros
Model fingerprint: 0x62c02722
Model has 10400 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-09, 1e+00]
  Objective range  [2e-03, 2e-03]
  QObjective range [2e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve time: 0.77s
Presolved: 400 rows, 10400 columns, 4000400 nonzeros
Presolved model has 10400 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.980e+04
 Factor NZ  : 8.020e+04 (roughly 5 MBytes of memory)
 Factor Ops : 2.141e+07 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.69866670e+09

In [45]:
T_per_cat = 2

2